<a href="https://colab.research.google.com/github/VitaliiSviatoshenko/duckdb/blob/main/%D0%9F%D1%80%D0%BE%D1%81%D1%83%D0%BD%D1%83%D1%82%D0%B0_%D1%80%D0%BE%D0%B1%D0%BE%D1%82%D0%B0_%D0%B7_%D1%82%D0%B0%D0%B1%D0%BB%D0%B8%D1%86%D1%8F%D0%BC%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandasql


  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26773 sha256=aa2143e77096168bf0e1b314c1381dd5fe9354aa74725214fb7b63d0b0b7760f
  Stored in directory: /root/.cache/pip/wheels/68/5d/a5/edc271b998f909801d7956959f699b976cc9896075dc47c153
Successfully built pandasql


In [2]:
import pandas as pd
from pandasql import sqldf

# Створюємо SQL-функцію
pysqldf = lambda q: sqldf(q, globals())


In [5]:
from google.colab import files

# Це відкриє форму вибору файлів – обери свої CSV-файли вручну
uploaded = files.upload()


Saving google_ads_basic_daily.csv to google_ads_basic_daily.csv


In [6]:
from google.colab import files

# Це відкриє форму вибору файлів – обери свої CSV-файли вручну
uploaded = files.upload()


Saving facebook_ads_basic_daily.csv to facebook_ads_basic_daily (1).csv


In [7]:
import pandas as pd

# Читаємо CSV у датафрейми
df_fb = pd.read_csv("facebook_ads_basic_daily.csv")
df_gg = pd.read_csv("google_ads_basic_daily.csv")

# Перевіряємо перші рядки
df_fb.head(), df_gg.head()


(      ad_date               campaign_id                  adset_id    spend  \
 0  2021-11-30  +1TVQ0Ksk5C1R7k9dKSrpg==  kjdNzjgMLmHu9QXPlKaQHQ==   2686.0   
 1  2022-04-12  pf4uuqaAAFuHa7gEJnQVKg==  +/ujgC+9ZiLznb2LxP6ihw==  25154.0   
 2  2021-06-17  +1TVQ0Ksk5C1R7k9dKSrpg==  QBLf/5REFjL80pmdlrLGCw==   4242.0   
 3  2022-01-31  h4hWTQf6icvBNjHfA+zKhw==  MVc+HmWqkmrlUI6Iyjdw4w==   1693.0   
 4  2022-04-05  pf4uuqaAAFuHa7gEJnQVKg==  +/ujgC+9ZiLznb2LxP6ihw==  45152.0   
 
    impressions    reach  clicks  leads    value  \
 0       5252.0   4624.0    13.0    NaN   4058.0   
 1       4578.0  16804.0   298.0    NaN  31921.0   
 2       6045.0   2323.0    97.0    0.0   5597.0   
 3      19554.0   3318.0    80.0    2.0   3383.0   
 4      19189.0  12587.0   244.0    7.0  53175.0   
 
                                       url_parameters  total  
 0  utm_source=facebook&utm_medium=ppc&utm_campaig...    NaN  
 1  utm_source=facebook&utm_medium=ppc&utm_campaig...    NaN  
 2                   

In [9]:
# Об’єднання двох датафреймів у CTE (Common Table Expression)
# Додаємо колонку media_source, щоб розрізняти джерела даних
query = """
WITH table_CTE AS (

    SELECT
        ad_date,
        'Facebook Ads' AS media_source,
        spend,
        impressions,
        reach,
        clicks,
        leads,
        value
    FROM df_fb

    UNION ALL

    SELECT
        ad_date,
        'Google Ads' AS media_source,
        spend,
        impressions,
        reach,
        clicks,
        leads,
        value
    FROM df_gg
)

SELECT
    ad_date,
    media_source,
    SUM(spend) AS spend,
    SUM(impressions) AS impressions,
    SUM(clicks) AS clicks,
    SUM(value) AS value
FROM table_CTE
GROUP BY ad_date, media_source
ORDER BY ad_date;
"""

result = pysqldf(query)
result.head()  # переглянути результат


,ad_date,media_source,spend,impressions,clicks,value
0,None,Facebook Ads,NaN,NaN,NaN,NaN
1,2020-11-11,Facebook Ads,189.0,1800.0,45.0,238.0
2,2020-11-12,Facebook Ads,2300.0,10473.0,397.0,2113.0
3,2020-11-12,Google Ads,398.0,9304.0,259.0,516.0
4,2020-11-13,Facebook Ads,636.0,11669.0,451.0,977.0
